In [ ]:
import os
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import logging
import warnings
import keras

from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, GlobalAveragePooling2D, Dropout, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D

logging.getLogger("tensorflow").setLevel(logging.ERROR)


In [ ]:

print(os.listdir())
train_path='/content/drive/MyDrive/Fillterd/train/'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['in', 'out'], batch_size=8, shuffle=True)

test_path='/content/drive/MyDrive/Fillterd/test/' 
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
   .flow_from_directory(directory=test_path, target_size=(224,224), classes=['in', 'out'], batch_size=8, shuffle=False)


In [ ]:

warnings.simplefilter("ignore")
model_name='EfficientNetB3'
base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", pooling='avg') 
# Note you are always told NOT to make the base model trainable initially- that is WRONG you get better results leaving it trainable
base_model.trainable=True
x=base_model.output
# 
# x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
# loss='binary_crossentropy' activation='sigmoid'
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(1024, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.3, seed=123)(x)
x = Dense(128, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.25, seed=123)(x)        
output=Dense(2, activation='sigmoid')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=.001 # start with this learning rate
model.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 
# model.compile(optimizer='adam',loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics=['accuracy'])

# rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,verbose=1)
# estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, verbose=1,restore_best_weights=True)
mc = ModelCheckpoint('efNetB32.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

callbacks=[mc]

epochs=10
history2=model.fit(x=train_batches,  epochs=epochs, callbacks=callbacks,  validation_data=None,
               validation_steps=None,  verbose=1, shuffle=True,  initial_epoch=0)


In [ ]:

# test_path='/content/drive/MyDrive/Fillterd/test/' 
# test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
#    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['in', 'out'], batch_size=16, shuffle=False)


from keras.models import load_model

saved_model = load_model('/content/drive/MyDrive/ComputerVision/efNetB3.h5')

test_loss, test_acc = saved_model.evaluate(test_batches, verbose=1)
print('Test: %.3f' % (test_acc))
print('Loss: %.3f' % (test_loss))

In [ ]:
# To Plot
import pickle
with open('/content/drive/MyDrive/ComputerVision/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history2.history, file_pi)

from keras.models import load_model

saved_model = load_model('/content/drive/MyDrive/ComputerVision/efNetB3.h5')

h=np.load('/content/drive/MyDrive/ComputerVision/trainHistoryDict',allow_pickle='TRUE')
# h

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter( y=h['val_loss'], name="test loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=h['loss'], name="train loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=h['val_accuracy'], name="test accuracy"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter( y=h['accuracy'], name="train accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss/Accuracy of EfficientNetB3 Model"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Loss</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Accuracy</b>", secondary_y=True)

fig.show()

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

pd.DataFrame(h).plot(figsize=(8,5))
plt.show()

# plt.plot(h['accuracy'])
# plt.plot(h['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper right')
# plt.show()

# plt.plot(h['loss'])
# plt.plot(h['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper right')
# plt.show()

***Segmentation***

In [ ]:
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random 
import torch
import glob
import cv2
import os


def getColor(id):
  if id ==0:
    return [0,255,255]
  elif id ==1:
    return [139,131,120]
  elif id ==2:
    return [227,207,87]
  elif id ==3:
    return [0,0,139]
  elif id ==4:
    return [139,35,35]
  elif id ==5:
    return [152,245,255]
  elif id ==6:
    return [127,255,0]
  elif id ==7:
    return [205,91,69]
  elif id ==8:
    return [0,205,205]
  elif id ==9:
    return [154,50,205]
  elif id ==10:
    return [205,16,118]
  elif id ==11:
    return [0,201,87]

model =torch.hub.load('ultralytics/yolov5','custom',path='/content/drive/MyDrive/ACV/models/best2.pt',force_reload=True)

model.conf = 0.30  # confidence threshold (0-1)
model.iou = 0.30  # NMS IoU threshold (0-1)

os.chdir('/content/drive/MyDrive/ACV/Test/Images/')
images = glob.glob('*')



In [ ]:

for img in images:
  e = model(img, size=640)

  original_image = cv2.imread(img)

  img2=cv2.cvtColor(original_image,cv2.COLOR_BGR2RGB)

  HSVimg = cv2.cvtColor(img2, cv2.COLOR_RGB2HSV)
  black = np.zeros(original_image.shape, dtype=np.uint8)

  bla=[0,0,0]
  for i in range(len(e.pandas().xyxy[0]['class'])):
    color = getColor(class_id)
    xmin = int(e.pandas().xyxy[0]['xmin'][i])
    xmax = int(e.pandas().xyxy[0]['xmax'][i])
    ymin = int(e.pandas().xyxy[0]['ymin'][i])
    ymax = int(e.pandas().xyxy[0]['ymax'][i])
    class_id = int(e.pandas().xyxy[0]['class'][i])
    subimg = HSVimg[ymin:ymax,xmin:xmax]
    vectorized = subimg.reshape((-1,3))
    vectorized = np.float32(vectorized)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 2
    attempts=10
    ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    # convert back to 8 bit values
    centers = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((subimg.shape))
    # flatten the labels array
    labels = label.flatten()

    # disable only the cluster number 2 (turn the pixel into black)
    masked_image = np.copy(subimg)
    # convert to the shape of a vector of pixel values
    masked_image = masked_image.reshape((-1, 3))
    # color (i.e cluster) to disable

    if str(e.pandas().xyxy[0]['name'][i])=='Bicycle':
      temp = color
      color = bla
      bla = temp
      # print('swap is done')
    df = pd.DataFrame(labels)
    if df.value_counts()[0] > df.value_counts()[1]:
      masked_image[labels == 0] = color
      masked_image[labels == 1] = bla
    else:
      masked_image[labels == 1] = color
      masked_image[labels == 0] = bla
    
    bla=[0,0,0]

    # convert back to original shape
    masked_image = masked_image.reshape(subimg.shape)
    # show the image

    black[ymin:ymax,xmin:xmax] = masked_image
    # cv2.imwrite('Severe%d.png'%(index),img)
  cv2.imwrite('/content/drive/MyDrive/ACV/Seg/'+img,black)
  # cv2_imshow(original_image)
  # cv2_imshow(black)